# CODICE BERT INIZIALE SULL'ITALIANO E CON TEST SPLIT

CODICE BERT INIZIALE SULL'ITALIANO E CON TEST SPLIT

In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

Mounted at /content/drive


In [ ]:
# Tokenizer di BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [ ]:
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles


In [ ]:
# Caricamento e divisione dei dati
texts, labels, titles = load_data(data_path)
train_texts, test_texts, train_labels, test_labels, train_titles, test_titles = train_test_split(texts, labels, titles, test_size=0.2, random_state=42)

# Creazione dei dataset
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

In [ ]:
# Configurazione dei parametri di training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)


In [ ]:
# Inizializzazione del modello e del trainer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training e valutazione del modello
trainer.train()
trainer.save_model("./best_model")  # Salva il modello con la migliore accuratezza


Epoch,Training Loss,Validation Loss,Accuracy
1,0.703100,0.709749,0.699029
2,0.675500,0.786750,0.708738
3,0.620500,0.590616,0.796117


In [ ]:
# Valutazione manuale sui dati di test
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)


In [ ]:
# Calcolo del classification report
target_names = ['0_bambini', '1_ragazzi', '2_adulti']
report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

# Stampa del classification report formattato
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    class_report = report[class_name]
    print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

print("\n\t\t\tSupport")
print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

# Funzione per la stampa degli errori usando il nome del file come titolo
def print_errors(titles, test_labels, predicted_labels, target_names, max_errors=10):
    print("\nErrori nei testi di test:")
    error_count = 0
    for i, (title, true_label, predicted_label) in enumerate(zip(titles, test_labels, predicted_labels)):
        if true_label != predicted_label:
            print(f"\nTitolo: {title}")
            print(f"Etichetta reale: {target_names[true_label]}")
            print(f"Etichetta predetta: {target_names[predicted_label]}")
            error_count += 1
            if error_count >= max_errors:
                break
    if error_count == 0:
        print("Nessun errore trovato.")

# Utilizza la funzione per la stampa degli errori
print_errors(test_titles, test_labels, predicted_labels, target_names)


		Precision	Recall	F1-score	Support
0_bambini	0.87		0.87	0.87		46
1_ragazzi	0.62		0.83	0.71		35
2_adulti	0.60		0.27	0.37		22

			Support
Accuracy		0.73	103
Macro avg	0.70		0.66	0.65		103
Weighted avg	0.73		0.73	0.71		103

Errori nei testi di test:

Titolo: ted2020-31.txt
Etichetta reale: 1_ragazzi
Etichetta predetta: 2_adulti

Titolo: YTP_003.txt
Etichetta reale: 2_adulti
Etichetta predetta: 0_bambini

Titolo: filmadulti46.txt
Etichetta reale: 2_adulti
Etichetta predetta: 0_bambini

Titolo: filmadulti42.txt
Etichetta reale: 2_adulti
Etichetta predetta: 1_ragazzi

Titolo: testo_ytkids_0031.txt
Etichetta reale: 0_bambini
Etichetta predetta: 1_ragazzi

Titolo: ted2020-127.txt
Etichetta reale: 1_ragazzi
Etichetta predetta: 2_adulti

Titolo: ted2020-101.txt
Etichetta reale: 1_ragazzi
Etichetta predetta: 2_adulti

Titolo: testo_ytkids_N030.txt
Etichetta reale: 1_ragazzi
Etichetta predetta: 0_bambini

Titolo: ted2020-183.txt
Etichetta reale: 2_adulti
Etichetta predetta: 1_ragazzi

Titolo: ted

# mBERT

# Mbert eng uncased

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/Esperimento_eng'

# Tokenizer di mBERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.020300,0.953747,0.688889
2,0.542900,0.625416,0.700000
3,0.354300,0.381404,0.866667


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.012200,0.935950,0.775281
2,0.579800,0.554301,0.820225
3,0.435800,0.328288,0.876404


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		40.0
1_ragazzi	0.75		1.00	0.86		33.0
2_adulti	1.00		0.31	0.48		16.0

			Support
Accuracy		0.88	89
Macro avg	0.92		0.77	0.78		89
Weighted avg	0.91		0.88	0.85		89

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.015700,0.927953,0.775281
2,0.586500,0.540399,0.786517
3,0.453400,0.320242,0.932584


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		37.0
1_ragazzi	0.85		1.00	0.92		33.0
2_adulti	1.00		0.68	0.81		19.0

			Support
Accuracy		0.93	89
Macro avg	0.95		0.89	0.91		89
Weighted avg	0.94		0.93	0.93		89

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		200.0
1_ragazzi	0.73		0.97	0.83		146.0
2_adulti	0.95		0.45	0.59		100.0

Macro avg	0.89		0.81	0.81
Weighted avg	0.90		0.87	0.86

Accuratezza media su 5 fold: 0.87


# mBERT ita uncased

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Tokenizer di mBERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.072000,0.999552,0.679612
2,0.829000,0.670833,0.747573
3,0.541900,0.579848,0.786408


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.066400,1.015423,0.660194
2,0.806000,0.745242,0.737864
3,0.467200,0.657697,0.747573


		Precision	Recall	F1-score	Support
0_bambini	0.85		0.79	0.82		43.0
1_ragazzi	0.63		0.94	0.76		36.0
2_adulti	1.00		0.38	0.55		24.0

			Support
Accuracy		0.75	103
Macro avg	0.83		0.70	0.71		103
Weighted avg	0.81		0.75	0.73		103

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.039100,0.991283,0.666667
2,0.776600,0.739408,0.686275
3,0.567800,0.639844,0.735294


		Precision	Recall	F1-score	Support
0_bambini	0.74		0.95	0.83		39.0
1_ragazzi	0.71		0.76	0.73		38.0
2_adulti	0.82		0.36	0.50		25.0

			Support
Accuracy		0.74	102
Macro avg	0.76		0.69	0.69		102
Weighted avg	0.75		0.74	0.71		102

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.77		0.90	0.82		209.0
1_ragazzi	0.67		0.80	0.73		179.0
2_adulti	0.79		0.33	0.45		126.0

Macro avg	0.74		0.68	0.67
Weighted avg	0.74		0.72	0.70

Accuratezza media su 5 fold: 0.72


# mbert cased ita

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Tokenizer di mBERT (cased)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.990500,0.962189,0.601942
2,0.836200,0.794913,0.699029
3,0.706700,0.953738,0.572816


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.007800,1.016464,0.417476
2,0.809700,0.782928,0.699029
3,0.542400,0.728135,0.699029


		Precision	Recall	F1-score	Support
0_bambini	0.75		0.95	0.84		43.0
1_ragazzi	0.65		0.61	0.63		36.0
2_adulti	0.64		0.38	0.47		24.0

			Support
Accuracy		0.70	103
Macro avg	0.68		0.65	0.65		103
Weighted avg	0.69		0.70	0.68		103

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.018600,0.938617,0.637255
2,0.788000,0.701932,0.735294
3,0.683100,0.787005,0.598039


		Precision	Recall	F1-score	Support
0_bambini	0.72		0.87	0.79		39.0
1_ragazzi	0.71		0.89	0.79		38.0
2_adulti	1.00		0.28	0.44		25.0

			Support
Accuracy		0.74	102
Macro avg	0.81		0.68	0.67		102
Weighted avg	0.79		0.74	0.70		102

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.78		0.91	0.84		209.0
1_ragazzi	0.64		0.80	0.71		179.0
2_adulti	0.44		0.21	0.28		126.0

Macro avg	0.62		0.64	0.61
Weighted avg	0.65		0.70	0.65

Accuratezza media su 5 fold: 0.70


# mbert cased ita badwords

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
import nltk

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Scarica risorse di NLTK per la tokenizzazione
nltk.download('punkt')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Caricamento delle bad words in italiano
bad_words_path = '/content/drive/My Drive/bad_words.txt'

try:
    with open(bad_words_path, 'r', encoding='utf-8') as file:
        bad_words = set(line.strip() for line in file)
except Exception as e:
    print(f"Error loading bad words: {e}")

# Funzione per rilevare bad words
def detect_bad_words(text, bad_words):
    words = set(word_tokenize(text.lower()))  # Tokenizza e abbassa il testo
    return words.intersection(bad_words)

# Tokenizer di mBERT (cased)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

# Lista per memorizzare i testi in cui l'etichetta è cambiata
texts_with_label_changes = []

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Aggiungi il controllo delle bad words
    adjusted_labels = []
    detected_bad_words_list = []

    for i, text in enumerate(test_texts):
        detected_bad_words = detect_bad_words(text, bad_words)
        original_label = predicted_labels[i]
        if original_label == 0 and detected_bad_words:
            adjusted_labels.append(2)  # Cambia l'etichetta in "adulti"
            detected_bad_words_list.append((titles[test_index[i]], detected_bad_words))
            # Memorizza i testi modificati
            texts_with_label_changes.append({
                "Title": titles[test_index[i]],
                "Original Text": text,
                "Detected Bad Words": list(detected_bad_words),
                "Original Label": original_label,  # Etichetta originale (0 = bambini)
                "New Label": 2  # Nuova etichetta (2 = adulti)
            })
        else:
            adjusted_labels.append(original_label)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, adjusted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

    # Stampa i testi in cui sono state rilevate bad words
    if detected_bad_words_list:
        print(f"Bad words rilevate nel fold {fold - 1}:")
        for title, bad_words in detected_bad_words_list:
            print(f"Titolo: {title}, Bad words: {bad_words}")

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")

# Stampa dei testi in cui l'etichetta è stata cambiata a causa delle bad words
if texts_with_label_changes:
    print(f"\nTesti con cambiamento di etichetta a causa di bad words ({len(texts_with_label_changes)} testi):")
    for item in texts_with_label_changes:
        print(f"Titolo: {item['Title']}")
        print(f"Testo originale: {item['Original Text']}")
        print(f"Parole inappropriate rilevate: {', '.join(item['Detected Bad Words'])}")
        print(f"Etichetta originale: {item['Original Label']}, Nuova etichetta: {item['New Label']}")
        print("-" * 80)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.990500,0.962189,0.601942
2,0.836200,0.794913,0.699029
3,0.706700,0.953738,0.572816


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.007800,1.016464,0.417476
2,0.809700,0.782928,0.699029
3,0.542400,0.728135,0.699029


		Precision	Recall	F1-score	Support
0_bambini	0.79		0.95	0.86		43.0
1_ragazzi	0.65		0.61	0.63		36.0
2_adulti	0.71		0.50	0.59		24.0

			Support
Accuracy		0.73	103
Macro avg	0.71		0.69	0.69		103
Weighted avg	0.72		0.73	0.72		103
Bad words rilevate nel fold 4:
Titolo: YTP_001.txt, Bad words: {'merda'}
Titolo: YTP_002.txt, Bad words: {'merda'}
Titolo: YTP_010.txt, Bad words: {'merda'}

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.018600,0.938617,0.637255
2,0.788000,0.701932,0.735294
3,0.683100,0.787005,0.598039


		Precision	Recall	F1-score	Support
0_bambini	0.81		0.87	0.84		39.0
1_ragazzi	0.71		0.89	0.79		38.0
2_adulti	1.00		0.48	0.65		25.0

			Support
Accuracy		0.78	102
Macro avg	0.84		0.75	0.76		102
Weighted avg	0.82		0.78	0.77		102
Bad words rilevate nel fold 5:
Titolo: YTP_006.txt, Bad words: {'merda'}
Titolo: YTP_021.txt, Bad words: {'merda'}
Titolo: YTP_022.txt, Bad words: {'merda'}
Titolo: filmadulti48.txt, Bad words: {'merda'}
Titolo: filmadulti44.txt, Bad words: {'merda'}

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.85		0.91	0.88		209.0
1_ragazzi	0.64		0.80	0.71		179.0
2_adulti	0.86		0.40	0.51		126.0

Macro avg	0.78		0.70	0.70
Weighted avg	0.78		0.74	0.72

Accuratezza media su 5 fold: 0.74

Testi con cambiamento di etichetta a causa di bad words (22 testi):
Titolo: YTP_003.txt
Testo originale: Gianni Morandi prego maestro
Vorrei che
cantassimo tutti insieme
fratelli d'Italia
vi piscio in testa
[Musica]
dio bestia
Dov'è la mia droga e porca ma Madonna
che 

# mbert cased eng

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/Esperimento_eng'

# Tokenizer di mBERT (cased)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.017700,0.935777,0.460674
2,0.378200,0.367005,0.865169
3,0.396200,0.294872,0.887640


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		40.0
1_ragazzi	0.77		1.00	0.87		33.0
2_adulti	1.00		0.38	0.55		16.0

			Support
Accuracy		0.89	89
Macro avg	0.92		0.79	0.80		89
Weighted avg	0.91		0.89	0.87		89

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.027100,0.901314,0.595506
2,0.515800,0.487314,0.820225
3,0.369400,0.274374,0.910112


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		37.0
1_ragazzi	0.82		0.97	0.89		33.0
2_adulti	0.92		0.63	0.75		19.0

			Support
Accuracy		0.91	89
Macro avg	0.91		0.87	0.88		89
Weighted avg	0.92		0.91	0.91		89

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		200.0
1_ragazzi	0.72		0.94	0.81		146.0
2_adulti	0.93		0.45	0.57		100.0

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		200.0
1_ragazzi	0.72		0.94	0.81		146.0
2_adulti	0.93		0.45	0.57		100.0

Macro avg	0.88		0.80	0.80
Weighted avg	0.89		0.87	0.85

Accuratezza media su 5 fold: 0.87


# mbert cased eng badwords

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
import nltk

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Scarica risorse di NLTK per la tokenizzazione
nltk.download('punkt')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/Esperimento_eng'

# Caricamento delle bad words in italiano
bad_words_path = '/content/drive/My Drive/badwords_eng_new.txt'

try:
    with open(bad_words_path, 'r', encoding='utf-8') as file:
        bad_words = set(line.strip() for line in file)
except Exception as e:
    print(f"Error loading bad words: {e}")
    bad_words = set()  # Definisci un set vuoto in caso di errore per evitare crash

# Funzione per rilevare bad words
def detect_bad_words(text, bad_words):
    words = set(word_tokenize(text.lower()))  # Tokenizza e abbassa il testo
    return words.intersection(bad_words)

# Tokenizer di mBERT (cased)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

# Lista per memorizzare i testi in cui l'etichetta è cambiata
texts_with_label_changes = []

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Aggiungi il controllo delle bad words
    adjusted_labels = []
    detected_bad_words_list = []

    for i, text in enumerate(test_texts):
        detected_bad_words = detect_bad_words(text, bad_words)
        original_label = predicted_labels[i]
        if original_label == 0 and detected_bad_words:
            adjusted_labels.append(2)  # Cambia l'etichetta in "adulti" se ci sono bad words
            detected_bad_words_list.append((titles[test_index[i]], detected_bad_words))

            # Memorizza i testi modificati
            texts_with_label_changes.append({
                "Title": titles[test_index[i]],
                "Original Text": text,
                "Detected Bad Words": list(detected_bad_words),
                "Original Label": original_label,  # Etichetta originale (0 = bambini)
                "New Label": 2  # Nuova etichetta (2 = adulti)
            })
        else:
            adjusted_labels.append(original_label)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, adjusted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

        # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

        # Stampa i testi in cui sono state rilevate bad words
    if detected_bad_words_list:
        print(f"Bad words rilevate nel fold {fold - 1}:")
        for title, bad_words in detected_bad_words_list:
            print(f"Titolo: {title}, Bad words: {bad_words}")

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")

# Stampa dei testi in cui l'etichetta è stata cambiata a causa delle bad words
if texts_with_label_changes:
    print(f"\nTesti con cambiamento di etichetta a causa di bad words ({len(texts_with_label_changes)} testi):")
    for item in texts_with_label_changes:
        print(f"Titolo: {item['Title']}")
        print(f"Testo originale: {item['Original Text']}")
        print(f"Parole inappropriate rilevate: {', '.join(item['Detected Bad Words'])}")
        print(f"Etichetta originale: {item['Original Label']}, Nuova etichetta: {item['New Label']}")
        print("-" * 80)


Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.017700,0.935777,0.460674
2,0.378200,0.367005,0.865169
3,0.396200,0.294872,0.887640


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		40.0
1_ragazzi	0.77		1.00	0.87		33.0
2_adulti	1.00		0.38	0.55		16.0

			Support
Accuracy		0.89	89
Macro avg	0.92		0.79	0.80		89
Weighted avg	0.91		0.89	0.87		89

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.027100,0.901314,0.595506
2,0.515800,0.487314,0.820225
3,0.369400,0.274374,0.910112


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		37.0
1_ragazzi	0.82		0.97	0.89		33.0
2_adulti	0.92		0.63	0.75		19.0

			Support
Accuracy		0.91	89
Macro avg	0.91		0.87	0.88		89
Weighted avg	0.92		0.91	0.91		89

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		200.0
1_ragazzi	0.72		0.93	0.81		146.0
2_adulti	0.93		0.46	0.58		100.0

Macro avg	0.88		0.80	0.80
Weighted avg	0.89		0.87	0.85

Accuratezza media su 5 fold: 0.87


# BERT

# bert uncased ita

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Tokenizer di BERT (inglese)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.892100,0.865728,0.563107
2,0.757100,0.738874,0.737864
3,0.594900,0.645248,0.737864


		Precision	Recall	F1-score	Support
0_bambini	0.83		0.91	0.87		43.0
1_ragazzi	0.65		0.86	0.74		36.0
2_adulti	0.75		0.25	0.38		24.0

			Support
Accuracy		0.74	103
Macro avg	0.74		0.67	0.66		103
Weighted avg	0.75		0.74	0.71		103

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.929000,0.864620,0.647059
2,0.833700,0.810057,0.637255
3,0.669100,0.720672,0.627451


		Precision	Recall	F1-score	Support
0_bambini	0.89		0.87	0.88		39.0
1_ragazzi	0.52		0.39	0.45		38.0
2_adulti	0.43		0.60	0.50		25.0

			Support
Accuracy		0.63	102
Macro avg	0.61		0.62	0.61		102
Weighted avg	0.64		0.63	0.63		102

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.87		0.90	0.89		209.0
1_ragazzi	0.59		0.71	0.64		179.0
2_adulti	0.68		0.40	0.46		126.0

Macro avg	0.71		0.67	0.66
Weighted avg	0.73		0.71	0.70

Accuratezza media su 5 fold: 0.71


# Bert uncased eng

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/Esperimento_eng'

# Tokenizer di BERT (inglese)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.986200,0.827604,0.711111
2,0.489400,0.497172,0.811111
3,0.279800,0.420436,0.866667


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.934400,0.760693,0.764045
2,0.508900,0.468639,0.842697
3,0.425200,0.368519,0.887640


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		40.0
1_ragazzi	0.77		1.00	0.87		33.0
2_adulti	1.00		0.38	0.55		16.0

			Support
Accuracy		0.89	89
Macro avg	0.92		0.79	0.80		89
Weighted avg	0.91		0.89	0.87		89

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.932300,0.807340,0.764045
2,0.518600,0.483763,0.876404
3,0.382600,0.296617,0.910112


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		37.0
1_ragazzi	0.84		0.94	0.89		33.0
2_adulti	0.87		0.68	0.76		19.0

			Support
Accuracy		0.91	89
Macro avg	0.90		0.87	0.88		89
Weighted avg	0.91		0.91	0.91		89

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.99		1.00	1.00		200.0
1_ragazzi	0.73		0.95	0.82		146.0
2_adulti	0.90		0.47	0.60		100.0

Macro avg	0.87		0.81	0.81
Weighted avg	0.89		0.87	0.86

Accuratezza media su 5 fold: 0.87


# Bert cased eng

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/Esperimento_eng'

# Tokenizer di BERT (versione cased)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer (versione cased)
    model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.992100,0.817409,0.808989
2,0.387600,0.388015,0.853933
3,0.422600,0.299795,0.876404


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		40.0
1_ragazzi	0.76		0.97	0.85		33.0
2_adulti	0.86		0.38	0.52		16.0

			Support
Accuracy		0.88	89
Macro avg	0.87		0.78	0.79		89
Weighted avg	0.89		0.88	0.86		89

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.991300,0.821279,0.786517
2,0.404100,0.356052,0.910112
3,0.358000,0.277880,0.887640


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		37.0
1_ragazzi	0.87		0.82	0.84		33.0
2_adulti	0.71		0.79	0.75		19.0

			Support
Accuracy		0.89	89
Macro avg	0.86		0.87	0.86		89
Weighted avg	0.89		0.89	0.89		89

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		200.0
1_ragazzi	0.74		0.92	0.81		146.0
2_adulti	0.88		0.49	0.59		100.0

Macro avg	0.87		0.80	0.80
Weighted avg	0.89		0.87	0.85

Accuratezza media su 5 fold: 0.87


# Bert cased eng badwords

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
import nltk

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Scarica risorse di NLTK per la tokenizzazione
nltk.download('punkt')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/Esperimento_eng'

# Caricamento delle bad words in inglese
bad_words_path = '/content/drive/My Drive/badwords_eng_new.txt'

try:
    with open(bad_words_path, 'r', encoding='utf-8') as file:
        bad_words = set(line.strip() for line in file)
except Exception as e:
    print(f"Error loading bad words: {e}")

# Funzione per rilevare bad words
def detect_bad_words(text, bad_words):
    words = set(word_tokenize(text.lower()))  # Tokenizza e abbassa il testo
    return words.intersection(bad_words)

# Tokenizer di BERT (versione cased)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer (versione cased)
    model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Aggiungi il controllo delle bad words
    adjusted_labels = []
    detected_bad_words_list = []

    for i, text in enumerate(test_texts):
        detected_bad_words = detect_bad_words(text, bad_words)
        original_label = predicted_labels[i]
        if original_label == 0 and detected_bad_words:
            adjusted_labels.append(2)  # Cambia l'etichetta in "adulti"
            detected_bad_words_list.append((titles[test_index[i]], detected_bad_words))
        else:
            adjusted_labels.append(original_label)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, adjusted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

    # Stampa i testi in cui sono state rilevate bad words
    if detected_bad_words_list:
        print(f"Bad words rilevate nel fold {fold - 1}:")
        for title, bad_words in detected_bad_words_list:
            print(f"Titolo: {title}, Bad words: {bad_words}")

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.978700,0.901758,0.700000
2,0.435900,0.464189,0.777778
3,0.263600,0.369538,0.855556


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.992100,0.817409,0.808989
2,0.387600,0.388015,0.853933
3,0.422600,0.299795,0.876404


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		40.0
1_ragazzi	0.76		0.97	0.85		33.0
2_adulti	0.86		0.38	0.52		16.0

			Support
Accuracy		0.88	89
Macro avg	0.87		0.78	0.79		89
Weighted avg	0.89		0.88	0.86		89

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.991300,0.821279,0.786517
2,0.404100,0.356052,0.910112
3,0.358000,0.277880,0.887640


		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		37.0
1_ragazzi	0.87		0.82	0.84		33.0
2_adulti	0.71		0.79	0.75		19.0

			Support
Accuracy		0.89	89
Macro avg	0.86		0.87	0.86		89
Weighted avg	0.89		0.89	0.89		89

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	1.00		1.00	1.00		200.0
1_ragazzi	0.74		0.92	0.81		146.0
2_adulti	0.88		0.49	0.59		100.0

Macro avg	0.87		0.80	0.80
Weighted avg	0.89		0.87	0.85

Accuratezza media su 5 fold: 0.87


# Bert cased ita

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Tokenizer di BERT (versione cased per inglese)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.146200,1.052214,0.466019
2,0.802600,0.733716,0.757282
3,0.659700,1.024455,0.592233


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.136700,1.064904,0.359223
2,0.757800,0.804839,0.718447
3,0.584500,0.704514,0.718447


		Precision	Recall	F1-score	Support
0_bambini	0.81		0.79	0.80		43.0
1_ragazzi	0.64		0.89	0.74		36.0
2_adulti	0.73		0.33	0.46		24.0

			Support
Accuracy		0.72	103
Macro avg	0.73		0.67	0.67		103
Weighted avg	0.73		0.72	0.70		103

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.094000,1.014906,0.578431
2,0.770800,1.011697,0.509804
3,0.675200,0.804774,0.598039


		Precision	Recall	F1-score	Support
0_bambini	0.74		0.95	0.83		39.0
1_ragazzi	0.62		0.13	0.22		38.0
2_adulti	0.43		0.76	0.55		25.0

			Support
Accuracy		0.60	102
Macro avg	0.60		0.61	0.53		102
Weighted avg	0.62		0.60	0.53		102

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.79		0.87	0.82		209.0
1_ragazzi	0.66		0.69	0.63		179.0
2_adulti	0.67		0.43	0.47		126.0

Macro avg	0.71		0.66	0.64
Weighted avg	0.72		0.70	0.67

Accuratezza media su 5 fold: 0.70


# Bert cased ita badwords

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
import nltk

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Scarica risorse di NLTK per la tokenizzazione
nltk.download('punkt')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Caricamento delle bad words in italiano
bad_words_path = '/content/drive/My Drive/bad_words.txt'  # Percorso corretto
try:
    with open(bad_words_path, 'r', encoding='utf-8') as file:
        bad_words = set(line.strip() for line in file)
except Exception as e:
    print(f"Error loading bad words: {e}")
    bad_words = set()  # Set vuoto in caso di errore

# Funzione per rilevare bad words
def detect_bad_words(text, bad_words):
    words = set(word_tokenize(text.lower()))  # Tokenizza e abbassa il testo
    return words.intersection(bad_words)

# Tokenizer di BERT (versione cased per inglese)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista per memorizzare i testi in cui l'etichetta è cambiata
texts_with_label_changes = []

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Aggiungi il controllo delle bad words
    adjusted_labels = []
    detected_bad_words_list = []

    for i, text in enumerate(test_texts):
        detected_bad_words = detect_bad_words(text, bad_words)
        original_label = predicted_labels[i]
        if original_label == 0 and detected_bad_words:
            adjusted_labels.append(2)  # Cambia l'etichetta in "adulti"
            detected_bad_words_list.append((titles[test_index[i]], detected_bad_words))

            # Memorizza i testi modificati
            texts_with_label_changes.append({
                "Title": titles[test_index[i]],
                "Original Text": text,
                "Detected Bad Words": list(detected_bad_words),
                "Original Label": original_label,  # Etichetta originale (0 = bambini)
                "New Label": 2  # Nuova etichetta (2 = adulti)
            })
        else:
            adjusted_labels.append(original_label)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, adjusted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

    # Stampa i testi in cui sono state rilevate bad words
    if detected_bad_words_list:
        print(f"Bad words rilevate nel fold {fold - 1}:")
        for title, bad_words in detected_bad_words_list:
            print(f"Titolo: {title}, Bad words: {', '.join(bad_words)}")
# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")

# Stampa dei testi in cui l'etichetta è stata cambiata a causa delle bad words
if texts_with_label_changes:
    print(f"\nTesti con cambiamento di etichetta a causa di bad words ({len(texts_with_label_changes)} testi):")
    for item in texts_with_label_changes:
        print(f"Titolo: {item['Title']}")
        print(f"Testo originale: {item['Original Text']}")
        print(f"Parole inappropriate rilevate: {', '.join(item['Detected Bad Words'])}")
        print(f"Etichetta originale: {item['Original Label']}, Nuova etichetta: {item['New Label']}")
        print("-" * 80)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.136700,1.064904,0.359223
2,0.757800,0.804839,0.718447
3,0.584500,0.704514,0.718447


		Precision	Recall	F1-score	Support
0_bambini	0.85		0.79	0.82		43.0
1_ragazzi	0.64		0.89	0.74		36.0
2_adulti	0.77		0.42	0.54		24.0

			Support
Accuracy		0.74	103
Macro avg	0.75		0.70	0.70		103
Weighted avg	0.76		0.74	0.73		103
Bad words rilevate nel fold 4:
Titolo: YTP_002.txt, Bad words: merda
Titolo: YTP_010.txt, Bad words: merda

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.094000,1.014906,0.578431
2,0.770800,1.011697,0.509804
3,0.675200,0.804774,0.598039


		Precision	Recall	F1-score	Support
0_bambini	0.79		0.95	0.86		39.0
1_ragazzi	0.62		0.13	0.22		38.0
2_adulti	0.47		0.88	0.61		25.0

			Support
Accuracy		0.63	102
Macro avg	0.63		0.65	0.56		102
Weighted avg	0.65		0.63	0.56		102
Bad words rilevate nel fold 5:
Titolo: YTP_006.txt, Bad words: merda
Titolo: YTP_021.txt, Bad words: merda
Titolo: YTP_022.txt, Bad words: merda

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.84		0.87	0.85		209.0
1_ragazzi	0.66		0.69	0.63		179.0
2_adulti	0.71		0.55	0.57		126.0

Macro avg	0.73		0.70	0.68
Weighted avg	0.75		0.72	0.70

Accuratezza media su 5 fold: 0.72

Testi con cambiamento di etichetta a causa di bad words (14 testi):
Titolo: YTP_003.txt
Testo originale: Gianni Morandi prego maestro
Vorrei che
cantassimo tutti insieme
fratelli d'Italia
vi piscio in testa
[Musica]
dio bestia
Dov'è la mia droga e porca ma Madonna
che schiava chiava
Dio porco
[Musica]
Ciao stronza ho deciso di scriverti una
lettera voglio farti una premess

# dbmdz

# dbmdz ita cased

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Tokenizer di BERT specifico per l'italiano
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-italian-cased')  # Puoi usare anche la versione uncased

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('dbmdz/bert-base-italian-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.009900,0.981162,0.601942
2,0.686100,0.811373,0.689320
3,0.602300,0.756578,0.689320


		Precision	Recall	F1-score	Support
0_bambini	0.78		0.84	0.81		43.0
1_ragazzi	0.61		0.83	0.71		36.0
2_adulti	0.62		0.21	0.31		24.0

			Support
Accuracy		0.69	103
Macro avg	0.67		0.63	0.61		103
Weighted avg	0.69		0.69	0.66		103

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.040200,0.965988,0.666667
2,0.730900,0.742987,0.676471
3,0.667800,0.684118,0.754902


		Precision	Recall	F1-score	Support
0_bambini	0.79		0.95	0.86		39.0
1_ragazzi	0.78		0.66	0.71		38.0
2_adulti	0.65		0.60	0.62		25.0

			Support
Accuracy		0.75	102
Macro avg	0.74		0.74	0.73		102
Weighted avg	0.75		0.75	0.75		102

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.82		0.89	0.85		209.0
1_ragazzi	0.64		0.81	0.70		179.0
2_adulti	0.77		0.30	0.40		126.0

Macro avg	0.74		0.67	0.65
Weighted avg	0.75		0.72	0.69

Accuratezza media su 5 fold: 0.72


# dbmdz ita cased badwords

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
import nltk

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Scarica risorse di NLTK per la tokenizzazione
nltk.download('punkt')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Caricamento delle bad words in italiano
bad_words_path = '/content/drive/My Drive/bad_words.txt'

try:
    with open(bad_words_path, 'r', encoding='utf-8') as file:
        bad_words = set(line.strip() for line in file)
except Exception as e:
    print(f"Error loading bad words: {e}")
    bad_words = set()  # Definisci un set vuoto in caso di errore per evitare crash

# Funzione per rilevare bad words
def detect_bad_words(text, bad_words):
    words = set(word_tokenize(text.lower()))  # Tokenizza e abbassa il testo
    return words.intersection(bad_words)

# Tokenizer di BERT specifico per l'italiano
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-italian-cased')  # Puoi usare anche la versione uncased

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista per memorizzare i testi in cui l'etichetta è cambiata
texts_with_label_changes = []

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    # Inizializzazione del modello e del trainer
    model = BertForSequenceClassification.from_pretrained('dbmdz/bert-base-italian-cased', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Aggiungi il controllo delle bad words
    adjusted_labels = []
    detected_bad_words_list = []

    for i, text in enumerate(test_texts):
        detected_bad_words = detect_bad_words(text, bad_words)
        original_label = predicted_labels[i]
        if original_label == 0 and detected_bad_words:
            adjusted_labels.append(2)  # Cambia l'etichetta in "adulti"
            detected_bad_words_list.append((titles[test_index[i]], detected_bad_words))

            # Memorizza i testi modificati
            texts_with_label_changes.append({
                "Title": titles[test_index[i]],
                "Original Text": text,
                "Detected Bad Words": list(detected_bad_words),
                "Original Label": original_label,  # Etichetta originale (0 = bambini)
                "New Label": 2  # Nuova etichetta (2 = adulti)
            })
        else:
            adjusted_labels.append(original_label)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, adjusted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

    # Stampa i testi in cui sono state rilevate bad words
    if detected_bad_words_list:
        print(f"Bad words rilevate nel fold {fold - 1}:")
        for title, bad_words in detected_bad_words_list:
            print(f"Titolo: {title}, Bad words: {bad_words}")

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")

# Stampa dei testi in cui l'etichetta è stata cambiata a causa delle bad words
if texts_with_label_changes:
    print(f"\nTesti con cambiamento di etichetta a causa di bad words ({len(texts_with_label_changes)} testi):")
    for item in texts_with_label_changes:
        print(f"Titolo: {item['Title']}")
        print(f"Testo originale: {item['Original Text']}")
        print(f"Parole inappropriate rilevate: {', '.join(item['Detected Bad Words'])}")
        print(f"Etichetta originale: {item['Original Label']}, Nuova etichetta: {item['New Label']}")
        print("-" * 80)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.015900,0.977863,0.679612
2,0.774600,0.703113,0.718447
3,0.658200,0.859408,0.621359


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.009900,0.981162,0.601942
2,0.686100,0.811373,0.689320
3,0.602300,0.756578,0.689320


		Precision	Recall	F1-score	Support
0_bambini	0.82		0.84	0.83		43.0
1_ragazzi	0.61		0.83	0.71		36.0
2_adulti	0.70		0.29	0.41		24.0

			Support
Accuracy		0.71	103
Macro avg	0.71		0.65	0.65		103
Weighted avg	0.72		0.71	0.69		103
Bad words rilevate nel fold 4:
Titolo: YTP_002.txt, Bad words: {'culo'}
Titolo: YTP_009.txt, Bad words: {'culo'}

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.040200,0.965988,0.666667
2,0.730900,0.742987,0.676471
3,0.667800,0.684118,0.754902


		Precision	Recall	F1-score	Support
0_bambini	0.80		0.95	0.87		39.0
1_ragazzi	0.78		0.66	0.71		38.0
2_adulti	0.67		0.64	0.65		25.0

			Support
Accuracy		0.76	102
Macro avg	0.75		0.75	0.75		102
Weighted avg	0.76		0.76	0.76		102
Bad words rilevate nel fold 5:
Titolo: YTP_022.txt, Bad words: {'culo'}

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.85		0.89	0.87		209.0
1_ragazzi	0.64		0.81	0.70		179.0
2_adulti	0.80		0.39	0.49		126.0

Macro avg	0.76		0.70	0.69
Weighted avg	0.77		0.74	0.72

Accuratezza media su 5 fold: 0.74

Testi con cambiamento di etichetta a causa di bad words (10 testi):
Titolo: YTP_003.txt
Testo originale: Gianni Morandi prego maestro
Vorrei che
cantassimo tutti insieme
fratelli d'Italia
vi piscio in testa
[Musica]
dio bestia
Dov'è la mia droga e porca ma Madonna
che schiava chiava
Dio porco
[Musica]
Ciao stronza ho deciso di scriverti una
lettera voglio farti una premessa
ho sesso
Ho deciso che mi viene da piangere
e non sai la gente mi conos

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.82		0.89	0.85		209.0
1_ragazzi	0.64		0.81	0.70		179.0
2_adulti	0.77		0.30	0.40		126.0

Macro avg	0.74		0.67	0.65
Weighted avg	0.75		0.72	0.69

Accuratezza media su 5 fold: 0.72

# ELECTRA

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Tokenizer specifico per ELECTRA in italiano
tokenizer = ElectraTokenizer.from_pretrained('dbmdz/electra-base-italian-xxl-cased-discriminator')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",  # Valutazione a ogni epoca
        save_strategy="no",  # Disabilita il salvataggio dei checkpoint intermedi
        load_best_model_at_end=False  # Non caricare il miglior modello alla fine
    )

    # Inizializzazione del modello e del trainer
    model = ElectraForSequenceClassification.from_pretrained('dbmdz/electra-base-italian-xxl-cased-discriminator', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, predicted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")

# Salva il modello finale
trainer.save_model(output_dir="./final_model")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at dbmdz/electra-base-italian-xxl-cased-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.052500,1.044491,0.592233
2,0.874200,0.766153,0.728155
3,0.639300,0.802128,0.631068


...

		Precision	Recall	F1-score	Support
0_bambini	0.81		0.98	0.88		43.0
1_ragazzi	0.68		0.83	0.75		36.0
2_adulti	0.86		0.25	0.39		24.0

			Support
Accuracy		0.76	103
Macro avg	0.78		0.69	0.67		103
Weighted avg	0.78		0.76	0.72		103

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at dbmdz/electra-base-italian-xxl-cased-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.070900,1.054137,0.480392
2,0.872900,0.791923,0.705882
3,0.707000,0.710740,0.794118


		Precision	Recall	F1-score	Support
0_bambini	0.75		1.00	0.86		39.0
1_ragazzi	0.80		0.84	0.82		38.0
2_adulti	1.00		0.40	0.57		25.0

			Support
Accuracy		0.79	102
Macro avg	0.85		0.75	0.75		102
Weighted avg	0.83		0.79	0.77		102

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.85		0.90	0.86		209.0
1_ragazzi	0.64		0.89	0.73		179.0
2_adulti	0.75		0.25	0.38		126.0

Macro avg	0.75		0.68	0.66
Weighted avg	0.76		0.73	0.70

Accuratezza media su 5 fold: 0.73


ValueError: You are trying to save a non contiguous tensor: `electra.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

# electra con badwords

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
import nltk

# Montaggio di Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Scarica risorse di NLTK per la tokenizzazione
nltk.download('punkt')

# Percorso della cartella dei dati su Google Drive
data_path = '/content/drive/My Drive/esperimento'

# Caricamento delle bad words in italiano
bad_words_path = '/content/drive/My Drive/bad_words.txt'

try:
    with open(bad_words_path, 'r', encoding='utf-8') as file:
        bad_words = set(line.strip() for line in file)
except Exception as e:
    print(f"Error loading bad words: {e}")
    bad_words = set()  # Definisci un set vuoto in caso di errore per evitare crash

# Funzione per rilevare bad words
def detect_bad_words(text, bad_words):
    words = set(word_tokenize(text.lower()))  # Tokenizza e converte tutto in minuscolo
    return words.intersection(bad_words)

# Tokenizer specifico per ELECTRA in italiano
tokenizer = ElectraTokenizer.from_pretrained('dbmdz/electra-base-italian-xxl-cased-discriminator')

# Classe Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Funzione per calcolare le metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Funzione per caricare i dati
def load_data(data_path):
    texts, labels, titles = [], [], []
    label_dict = {'0_bambini': 0, '1_ragazzi': 1, '2_adulti': 2}
    for label, index in label_dict.items():
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(index)
                titles.append(filename)  # Salva il nome del file come titolo
    return texts, labels, titles

# Caricamento dei dati
texts, labels, titles = load_data(data_path)

# Inizializza dizionari per raccogliere i risultati medi
precision_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
recall_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
f1_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}
support_sum = {class_name: 0 for class_name in ['0_bambini', '1_ragazzi', '2_adulti']}

accuracy_per_fold = []
macro_avg_precision_sum, macro_avg_recall_sum, macro_avg_f1_sum = 0, 0, 0
weighted_avg_precision_sum, weighted_avg_recall_sum, weighted_avg_f1_sum = 0, 0, 0

fold = 1

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Lista per memorizzare i testi in cui l'etichetta è cambiata
texts_with_label_changes = []

for train_index, test_index in kfold.split(texts):
    print(f'\nFold {fold}')
    fold += 1

    # Divisione in train e test per il fold corrente
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]

    # Creazione dei dataset
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)

    # Configurazione dei parametri di training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",  # Valutazione a ogni epoca
        save_strategy="no",  # Disabilita il salvataggio dei checkpoint intermedi
        load_best_model_at_end=False  # Non caricare il miglior modello alla fine
    )

    # Inizializzazione del modello e del trainer
    model = ElectraForSequenceClassification.from_pretrained('dbmdz/electra-base-italian-xxl-cased-discriminator', num_labels=3)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Training del modello
    trainer.train()

    # Valutazione sul test del fold corrente
    predictions = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions.predictions, axis=1)

    # Aggiungi il controllo delle bad words
    adjusted_labels = []
    detected_bad_words_list = []

    for i, text in enumerate(test_texts):
        detected_bad_words = detect_bad_words(text, bad_words)
        original_label = predicted_labels[i]
        if original_label == 0 and detected_bad_words:
            adjusted_labels.append(2)  # Cambia l'etichetta in "adulti" se ci sono bad words
            detected_bad_words_list.append((titles[test_index[i]], detected_bad_words))

            # Memorizza i testi modificati
            texts_with_label_changes.append({
                "Title": titles[test_index[i]],
                "Original Text": text,
                "Detected Bad Words": list(detected_bad_words),
                "Original Label": original_label,  # Etichetta originale (0 = bambini)
                "New Label": 2  # Nuova etichetta (2 = adulti)
            })
        else:
            adjusted_labels.append(original_label)

    # Calcolo del classification report
    target_names = ['0_bambini', '1_ragazzi', '2_adulti']
    report = classification_report(test_labels, adjusted_labels, target_names=target_names, output_dict=True)

    # Stampa del classification report formattato
    print("\t\tPrecision\tRecall\tF1-score\tSupport")
    for class_name in target_names:
        class_report = report[class_name]
        print(f"{class_name}\t{class_report['precision']:.2f}\t\t{class_report['recall']:.2f}\t{class_report['f1-score']:.2f}\t\t{class_report['support']}")

    print("\n\t\t\tSupport")
    print(f"Accuracy\t\t{report['accuracy']:.2f}\t{len(test_labels)}")
    print(f"Macro avg\t{report['macro avg']['precision']:.2f}\t\t{report['macro avg']['recall']:.2f}\t{report['macro avg']['f1-score']:.2f}\t\t{len(test_labels)}")
    print(f"Weighted avg\t{report['weighted avg']['precision']:.2f}\t\t{report['weighted avg']['recall']:.2f}\t{report['weighted avg']['f1-score']:.2f}\t\t{len(test_labels)}")

    # Somma i valori di precision, recall, f1 e support per ogni classe
    for class_name in target_names:
        precision_sum[class_name] += report[class_name]['precision']
        recall_sum[class_name] += report[class_name]['recall']
        f1_sum[class_name] += report[class_name]['f1-score']
        support_sum[class_name] += report[class_name]['support']

    # Somma per macro avg e weighted avg
    macro_avg_precision_sum += report['macro avg']['precision']
    macro_avg_recall_sum += report['macro avg']['recall']
    macro_avg_f1_sum += report['macro avg']['f1-score']

    weighted_avg_precision_sum += report['weighted avg']['precision']
    weighted_avg_recall_sum += report['weighted avg']['recall']
    weighted_avg_f1_sum += report['weighted avg']['f1-score']

    # Aggiungi l'accuratezza alla lista per calcolare la media finale
    accuracy_per_fold.append(report['accuracy'])

    # Stampa i testi in cui sono state rilevate bad words
    if detected_bad_words_list:
        print(f"Bad words rilevate nel fold {fold - 1}:")
        for title, bad_words in detected_bad_words_list:
            print(f"Titolo: {title}, Bad words: {bad_words}")

# Media dei risultati per ogni classe
print("\nRisultati medi su 5 fold:")
print("\t\tPrecision\tRecall\tF1-score\tSupport")
for class_name in target_names:
    print(f"{class_name}\t{(precision_sum[class_name]/5):.2f}\t\t{(recall_sum[class_name]/5):.2f}\t{(f1_sum[class_name]/5):.2f}\t\t{support_sum[class_name]}")

# Media per Macro avg e Weighted avg
print(f"\nMacro avg\t{(macro_avg_precision_sum/5):.2f}\t\t{(macro_avg_recall_sum/5):.2f}\t{(macro_avg_f1_sum/5):.2f}")
print(f"Weighted avg\t{(weighted_avg_precision_sum/5):.2f}\t\t{(weighted_avg_recall_sum/5):.2f}\t{(weighted_avg_f1_sum/5):.2f}")

# Stampa dell'accuratezza media finale
print(f"\nAccuratezza media su 5 fold: {np.mean(accuracy_per_fold):.2f}")

# Stampa dei testi in cui l'etichetta è stata cambiata a causa delle bad words
if texts_with_label_changes:
    print(f"\nTesti con cambiamento di etichetta a causa di bad words ({len(texts_with_label_changes)} testi):")
    for item in texts_with_label_changes:
        print(f"Titolo: {item['Title']}")
        print(f"Testo originale: {item['Original Text']}")
        print(f"Parole inappropriate rilevate: {', '.join(item['Detected Bad Words'])}")
        print(f"Etichetta originale: {item['Original Label']}, Nuova etichetta: {item['New Label']}")
        print("-" * 80)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at dbmdz/electra-base-italian-xxl-cased-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.069600,1.046074,0.669903
2,0.880300,0.761209,0.737864
3,0.629700,0.764629,0.718447


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.067100,1.054812,0.621359
2,0.856900,0.812326,0.669903
3,0.556300,0.605449,0.757282


		Precision	Recall	F1-score	Support
0_bambini	0.82		0.98	0.89		43.0
1_ragazzi	0.68		0.83	0.75		36.0
2_adulti	0.88		0.29	0.44		24.0

			Support
Accuracy		0.77	103
Macro avg	0.79		0.70	0.69		103
Weighted avg	0.79		0.77	0.74		103
Bad words rilevate nel fold 4:
Titolo: YTP_001.txt, Bad words: {'cazzi'}

Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at dbmdz/electra-base-italian-xxl-cased-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.070900,1.054137,0.480392
2,0.872900,0.791923,0.705882
3,0.707000,0.710740,0.794118


		Precision	Recall	F1-score	Support
0_bambini	0.76		1.00	0.87		39.0
1_ragazzi	0.80		0.84	0.82		38.0
2_adulti	1.00		0.44	0.61		25.0

			Support
Accuracy		0.80	102
Macro avg	0.85		0.76	0.77		102
Weighted avg	0.84		0.80	0.79		102
Bad words rilevate nel fold 5:
Titolo: YTP_016.txt, Bad words: {'cazzi'}

Risultati medi su 5 fold:
		Precision	Recall	F1-score	Support
0_bambini	0.87		0.91	0.88		209.0
1_ragazzi	0.65		0.89	0.74		179.0
2_adulti	0.96		0.36	0.52		126.0

Macro avg	0.82		0.72	0.71
Weighted avg	0.82		0.76	0.74

Accuratezza media su 5 fold: 0.76

Testi con cambiamento di etichetta a causa di bad words (5 testi):
Titolo: YTP_012.txt
Testo originale: [Musica]
[Musica]
non è possibile non è matematicamente
Basta basta basta
Ecco gli amici di Peppa
[rumore di esplosione] 
Ognuno di loro ha una letterina per Babbo Natale
[Musica]
che cazzo avete chiesto a Babbo Bastardo?
una navicella aziale
una chitarra a giocare
Totò
una trombetta
una racchetta con la racchetta
George ha chiesto un
coltel

In [ ]:
# Ricarica il file delle bad words
bad_words_path = '/content/drive/My Drive/badwords_ita.txt'

try:
    with open(bad_words_path, 'r', encoding='utf-8') as file:
        bad_words = set(line.strip() for line in file)
        print(f"Bad words caricate correttamente: {len(bad_words)} parole.")
except Exception as e:
    print(f"Error loading bad words: {e}")
    bad_words = set()  # Definisci un set vuoto in caso di errore per evitare crash


Error loading bad words: [Errno 2] No such file or directory: '/content/drive/My Drive/badwords_ita.txt'
